In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request, json 
import requests

# Project Goal

Raiding is stressful enough, but loot just dropped and people are fighting/arguing what is BiS (Best in Slot) for them vs someone else -- That makes things REAL stressful!

No one person can be expected to the BiS rankings of <ins>every</ins> item dropped in raid to each cooresponding class/spec! (Not even multiple people can likely fragment that information between them).

##### Goal:
> Create dictionaries that can help Raid leaders/Loot Masters determine, at a glance, who are the BiS users when an item drops in raid

What this code will do:
1. Compile information from various BiS websites
2. Rank items from "1" to "X" based on the respective website's BiS ranking indicator (Ex: wowtbc.gg has "final_value" where a larger value indicates that item being better than another item of the same slot/spec/class)
3. Have 2 dictionaries:
> a) Dicitionary indicating BiS items for a given Class/Spec
> <br>b) Dictionary indicating the class/spec ranking for a given item (reverse of A)
4. Write these dictionaries to `.csv`
5. Create an amazing Google Sheet utilizing this information to allow Raid Leaders + Loot Masters to make educated decisions when it comes to distribution of loot based on their guild's loot rules (Ex: MS > OS, No Loot over Loot)

# wowtbc.gg

Scrap the idea for wowtbc.gg for now --- Injection of "T4" would not refresh the page with the updated info

In [4]:
#Populate the different URLs to access

wowtbc_prefix = r"https://wowtbc.gg/page-data/wotlk/bis-list/" #Wotlk update 01Oct22
#wowtbc_prefix = r"https://wowtbc.gg/page-data/wotlk/bis-list/"
wowtbc_suffix = r"/page-data.json"

class_and_specs = {"hunter":['beast-mastery','survival','marksmanship'],
                   "mage":['fire','arcane','frost'],
                   'priest':['holy','shadow','discipline'],
                   'warrior':['arms','fury','protection'],
                   'paladin':['holy','protection','retribution'],
                   "druid":['balance','restoration','feral-dps','feral-tank'],
                   "rogue":['combat','assassination'],
                   "shaman":['elemental','enhancement','restoration'],
                   "warlock":['affliction','demonology','destruction'],
                   "Death-Knight":['Unholy','Frost','Blood-Tank']}

urls = []
for key,val in class_and_specs.items():
    for spec in val:
        urls.append(wowtbc_prefix+spec+"-"+key+wowtbc_suffix)
        
urls

['https://wowtbc.gg/page-data/wotlk/bis-list/beast-mastery-hunter/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/survival-hunter/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/marksmanship-hunter/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/fire-mage/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/arcane-mage/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/frost-mage/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/holy-priest/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/shadow-priest/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/discipline-priest/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/arms-warrior/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/fury-warrior/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/protection-warrior/page-data.json',
 'https://wowtbc.gg/page-data/wotlk/bis-list/holy-paladin/page-data.json',
 '

Code to try and extract the json from the web url below ... it doesn't work ... keeps interpreting the "content-type" as "text/html" when it should be "application/json"

Backup solution -- manually download all json files and load them into memory instead =/

In [2]:
#Make a dictionary of all the Tier pieces and their related conversions
tier_dict = {
    "Helm of the Lost Conqueror":[
        "Heroes' Circlet of Faith",
        "Heroes' Crown of Faith",
        "Heroes' Plagueheart Circlet",
        "Heroes' Redemption Headpiece",
        "Heroes' Redemption Helm",
        "Heroes' Redemption Faceguard",
        ],
    "Chestguard of the Lost Conqueror":[
        "Heroes' Plagueheart Robe",
        "Heroes' Robe of Faith",
        "Heroes' Raiments of Faith",
        "Heroes' Redemption Tunic",
        "Heroes' Redemption Chestpiece",
        "Heroes' Redemption Breastplate",
        ],
    "Mantle of the Lost Conqueror":[
        "Valorous Plagueheart Shoulderpads",
        "Valorous Shoulderpads of Faith",
        "Valorous Mantle of Faith",
        "Valorous Redemption Spaulders",
        "Valorous Redemption Shoulderplates",
        "Valorous Redemption Shoulderguards",
    ],
    "Gloves of the Lost Conqueror":[
        "Heroes' Plagueheart Gloves",
        "Heroes' Gloves of Faith",
        "Heroes' Handwraps of Faith",
        "Heroes' Redemption Gloves",
        "Heroes' Redemption Gauntlets",
        "Heroes' Redemption Handguards",
    ],
    "Crown of the Lost Conqueror":[
        "Valorous Plagueheart Circlet",
        "Valorous Crown of Faith",
        "Valorous Circlet of Faith",
        "Valorous Redemption Headpiece",
        "Valorous Redemption Helm",
        "Valorous Redemption Faceguard",
    ],
    "Breastplate of the Lost Conqueror":[
        "Valorous Plagueheart Robe",
        "Valorous Robe of Faith",
        "Valorous Raiments of Faith",
        "Valorous Redemption Tunic",
        "Valorous Redemption Chestpiece",
        "Valorous Redemption Breastplate",
    ],
    "Legplates of the Lost Conqueror":[
        "Valorous Plagueheart Leggings",
        "Valorous Leggings of Faith",
        "Valorous Pants of Faith",
        "Valorous Redemption Greaves",
        "Valorous Redemption Legplates",
        "Valorous Redemption Legguards",
    ],
    "Gauntlets of the Lost Conqueror":[
        "Valorous Plagueheart Gloves",
        "Valorous Gloves of Faith",
        "Valorous Handwraps of Faith",
        "Valorous Redemption Gloves",
        "Valorous Redemption Gauntlets",
        "Valorous Redemption Handguards",
    ],
    "Spaulders of the Lost Conqueror":[
        "Heroes' Plagueheart Shoulderpads",
        "Heroes' Shoulderpads of Faith",
        "Heroes' Mantle of Faith",
        "Heroes' Redemption Spaulders",
        "Heroes' Redemption Shoulderplates",
        "Heroes' Redemption Shoulderguards",
    ],
    "Leggings of the Lost Conqueror":[
        "Heroes' Plagueheart Leggings",
        "Heroes' Leggings of Faith",
        "Heroes' Pants of Faith",
        "Heroes' Redemption Greaves",
        "Heroes' Redemption Legplates",
        "Heroes' Redemption Legguards",
    ],
    "Spaulders of the Wayward Conqueror":[
        "Valorous Aegis Spaulders",
        "Valorous Aegis Shoulderplates",
        "Valorous Aegis Shoulderguards",
    ],
    "Crown of the Wayward Conqueror":[
        "Conqueror's Aegis Helm",
        "Conqueror's Aegis Faceguard",
        "Conqueror's Aegis Headpiece",
    ],
    "Mantle of the Wayward Conqueror":[
        "Conqueror's Aegis Shoulderplates",
        "Conqueror's Aegis Shoulderguards",
        "Conqueror's Aegis Spaulders",
    ],
    "Helm of the Wayward Conqueror":[
        "Valorous Aegis Headpiece",
        "Valorous Aegis Helm",
        "Valorous Aegis Faceguard",
    ],
    "Breastplate of the Wayward Conqueror":[
        "Conqueror's Aegis Battleplate",
        "Conqueror's Aegis Breastplate",
        "Conqueror's Aegis Tunic",
    ],
    "Legplates of the Wayward Conqueror":[
        "Conqueror's Aegis Legplates",
        "Conqueror's Aegis Legguards",
        "Conqueror's Aegis Greaves",
    ],
    "Leggings of the Wayward Conqueror":[
        "Valorous Aegis Greaves",
        "Valorous Aegis Legplates",
        "Valorous Aegis Legguards",
    ],
    "Gloves of the Wayward Conqueror":[
        "Valorous Aegis Gloves",
        "Valorous Aegis Gauntlets",
        "Valorous Aegis Handguards",
    ],
    "Gauntlets of the Wayward Conqueror":[
        "Conqueror's Aegis Gauntlets",
        "Conqueror's Aegis Handguards",
        "Conqueror's Aegis Gloves",
    ],
    "Chestguard of the Wayward Conqueror":[
        "Valorous Aegis Tunic",
        "Valorous Aegis Battleplate",
        "Valorous Aegis Breastplate",
    ],
    }

#Convert dictionary to pd DataFrame
tier_df = pd.DataFrame({"Token":tier_dict.keys(),"Options":tier_dict.values()})

def tier_piece_items(item):
    '''
    Function to reverse the tier piece back into its Token name
    Ex: "Pauldrons of the Aldor" --> "Pauldrons of the Fallen Hero"
    '''
    token = item['name'] #Initialize token name as the item name
    for i, row in enumerate(tier_df['Options']): #Find item name in tier registry
        if item['name'] in row:
            token = tier_df['Token'][i] #If tier piece, replace token name
            break
    return token

Sartharion drop table: https://www.wowhead.com/wotlk/guides/raids/obsidian-sanctum/sartharion-strategy

token list: https://www.wowhead.com/wotlk/items/miscellaneous/armor-tokens/min-req-level:80/class:5

In [5]:
%%time
#Grab all files
import sys
import glob
expa = "wowwotlk"
files = glob.glob(expa+'/*.json')

#Hyperparameters
website = 'wowtbc.gg' #In the future, will add other websites
current_phase = "T7" #Change this in the future to "T6"
#raids = ['Karazhan',"Gruul's Lair","Magtheridon's Lair","SSC","Tempest Keep"]  #Zul'Aman

#'Ulduar','Trial of the Crusader','Onyxia''s Lair','Icecrown Citadel'
#raids = ['Vault of Archavon','Naxxramas','Obsidian Sanctum','Eye of Eternity'] 

nax_bosses = ["Anub'Rekhan","The Four Horsemen","Gluth","Grand Widow Faerlina","Maexxna",
            "Noth the Plaguebringer","Heigan the Unclean","Loatheb","Patchwerk",
            "Grobbulus","Thaddius","Instructor Razuvious","Gothik the Harvester",
            "Sapphiron","Kel'Thuzad",]
vault_bosses = ["Archavon the Stone Watcher","Emalon the Storm Watcher",
                "Koralon the Flame Watcher","Toravon the Ice Watcher"]
Sartharion_loot_dic = {
    "The Sanctum's Flowing Vestments" : "Obsidian Sanctum (25) - Sartharion 1 Drake",
    "Unsullied Cuffs" : "Obsidian Sanctum (25) - Sartharion 2 Drake",
    "Headpiece of Reconciliation" : "Obsidian Sanctum (25) - Sartharion 2 Drake",
    "Leggings of the Honored" : "Obsidian Sanctum (25) - Sartharion 2 Drake",
    "Concealment Shoulderpads" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Mantle of the Eternal Sentinel" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Bountiful Gauntlets" : "Obsidian Sanctum (25) - Sartharion 1 Drake",
    "Upstanding Spaulders" : "Obsidian Sanctum (25) - Sartharion 1 Drake",
    "Council Chamber Epaulets" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Hyaline Helm of the Sniper" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Dragonstorm Breastplate" : "Obsidian Sanctum (25) - Sartharion 1 Drake",
    "Obsidian Greathelm" : "Obsidian Sanctum (25) - Sartharion 2 Drake",
    "Chestplate of the Great Aspects" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Dragon Brood Legguards" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Pennant Cloak" : "Obsidian Sanctum (25) - Sartharion 2 Drake",
    "Wyrmrest Band" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Fury of the Five Flights" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Illustration of the Dragon Soul" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Staff of Restraint" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Gauntlets of the Lost Conqueror" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Gauntlets of the Lost Protector" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Gauntlets of the Lost Vanquisher" : "Obsidian Sanctum (25) - Sartharion 0 Drake",
    "Enamored Cowl" : "Obsidian Sanctum (10) - Sartharion 2 Drake",
    "Blade-Scarred Tunic" : "Obsidian Sanctum (10) - Sartharion 1 Drake",
    "Legguards of Composure" : "Obsidian Sanctum (10) - Sartharion 1 Drake",
    "Chestguard of Flagrant Prowess" : "Obsidian Sanctum (10) - Sartharion 2 Drake",
    "Sabatons of Firmament" : "Obsidian Sanctum (10) - Sartharion 2 Drake",
    "Belabored Legplates" : "Obsidian Sanctum (10) - Sartharion 2 Drake",
    "Remembrance Girdle" : "Obsidian Sanctum (10) - Sartharion 1 Drake",
    "Titan's Outlook" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Gale-Proof Cloak" : "Obsidian Sanctum (10) - Sartharion 1 Drake",
    "Greatring of Collision" : "Obsidian Sanctum (10) - Sartharion 2 Drake",
    "Signet of the Accord" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Majestic Dragon Figurine" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Circle of Arcane Streams" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Volitant Amulet" : "Obsidian Sanctum (10) - Sartharion 1 Drake",
    "Crimson Steel" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Gauntlets of the Lost Conqueror" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Gauntlets of the Lost Protector" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Gauntlets of the Lost Vanquisher" : "Obsidian Sanctum (10) - Sartharion 0 Drake",
    "Circle of Arcane Streams" : "Obsidian Sanctum (10) - Sartharion 0 Drake(?)",
    }
raids = ['Vault of Archavon','Naxxramas']
raid_boss_ref = [vault_bosses,nax_bosses]
raid_size = [" (10)"," (25)"]
raid_boss_registry = {}
for i,raid in enumerate(raids):
    for size in raid_size:
        for boss in raid_boss_ref[i]:
            raid_boss_registry[boss + size] = raids[i] + size
raids.append("Eye of Eternity (10)")
raids.append("Eye of Eternity (25)")
raids.append("Obsidian Sanctum (10)")
raids.append("Obsidian Sanctum (25)")

all_items = []
#---Big processing loop
print("Reading files:")
df = pd.DataFrame(columns=["Class","Spec","ItemName","Slot","Rank","Source","Source Type","Website"])
phases = []
for file in files:
    with open(file,) as json_file: #Open a file
        print("\t%s"%file)
        
#        if file == r"wowtbc\holy-priest.json":
#            break
        
        #Grab file name to obtain the Class and Spec information
        class_spec_str = file[9:-5].split("-")
        c = class_spec_str[-1]
        spec = ' '.join(class_spec_str[:-1])
        
        data = json.load(json_file) #Load the data as json
        
        #Initialize
        slot_count = 0
        slot_lock = "head"
        #Parse each item
        items = data['result']['pageContext']['bisList']
        all_items.append(items)
        for item in items:
            if 'name' in item.keys():
                item_name = item['name']
                if current_phase in item['phase']: #Make sure the item is currently obtainable

                    #Check item slot because there are some duplicate entries
                    if item['slot'] in ['trinket 1','trinket 2']: #Simplify "Trinket1" or "Trinket2" to "trinket"
                        slot = "trinket"
                    elif item['slot'] in ['finger 1','finger 2']: #Simplify "finger 1" or "finger 2" to "finger"
                        slot = "finger"
                    else:
                        slot = item['slot']                        

    #                print(slot,slot_lock,slot_count)
                    #Count the top N per a specific slot
                    if slot_lock == slot:
                        slot_count += 1 #Increment
                        if slot_lock in ['trinket','finger']:
                            X = 9 #Provide 1 more entries if trinket or finger because you can have 2 each
                        else:
                            X = 8
                        if slot_count <= X: #This limits search to only top X
    #                        print("%s registered"%item['name'])
                            #Check if item source 
                            try:
                                if item['source'] == 'Reputation':
                                    source = "Rep - " + item['source_type'] #If rep, add faction
                                elif item['source'] == "Quest Reward":
                                    source = "Quest - " + item['source_type'] #If quest, add quest name
#                                elif item['source'] == 'Vendor':
#                                    source = item['']
#                                elif item['source'] == "Multiple":
#                                    source = item['source_type']
#                                elif item['source'] in raids:
#                                    source = item['source'] + " - " + item['source_type']                                    
                                else:                            
                                    source = item['source']
                            except:
                                source = "Not found"

                            #Source type
                            try:
                                source_type = item['source_type']
                                if item['source_type'] == "Trash Drop":
                                    source_type = str(item['drop_chance']*100)+"%"
                                    source = item['source'] + " - " + item['source_type']
                                if item_name in Sartharion_loot_dic.keys():
                                    source = Sartharion_loot_dic[item_name].split(" - ")[0]
                                    source_type = Sartharion_loot_dic[item_name].split(" - ")[1]
                            except:
                                source_type = "NA"
                            
                            item_name = tier_piece_items(item)
                            
                            d = pd.DataFrame({"Class":[c],
                                              "Spec":[spec],
                                              "ItemName":[item_name],
                                              "Slot":[slot],
                                              "Rank":[slot_count],
                                              "Source": [source],
                                              "Source Type": [source_type],
                                              "Website":[website]})
                            df = pd.concat([df,d])
                        else:
                            pass #Don't do anything if slot_count > X
                    else:
                        slot_count = 1 #Reset
                        #Check if Slot is "Trinket1" or "Trinket2" and simplify to "trinket"
                        if item['slot'] in ['trinket 1','trinket 2']:
                            slot_lock = "trinket"
                            slot = 'trinket'
                        #Check if Slot is "finger 1" or "finger 2" and simplify to "finger"
                        elif item['slot'] in ['finger 1','finger 2']:
                            slot_lock = "finger"
                            slot = 'finger'
                        else:
                            slot_lock = item['slot'] #Set new lock parameter
                            slot = item['slot']

                        #Check if item source 
                        try:
                            if item['source'] == 'Reputation':
                                source = "Rep - " + item['source_type'] #If rep, add faction
                            elif item['source'] == "Quest Reward":
                                source = "Quest - " + item['source_type'] #If quest, add quest name
#                            elif item['source'] == 'Vendor':
#                                source = item['']
#                                elif item['source'] == "Multiple":
#                                    source = item['source_type']
#                                elif item['source'] in raids:
#                                    source = item['source'] + " - " + item['source_type']                                    
                            else:                            
                                source = item['source']
                        except:
                            source = "Not found"

                        #Source type
                        try:
                            source_type = item['source_type']
                            if item['source_type'] == "Trash Drop":
                                source_type = str(item['drop_chance']*100)+"%"
                                source = item['source'] + " - " + item['source_type']
                            if item_name in Sartharion_loot_dic.keys():
                                source = Sartharion_loot_dic[item_name].split(" - ")[0]
                                source_type = Sartharion_loot_dic[item_name].split(" - ")[1]
                        except:
                            source_type = "NA"                       
                        item_name = tier_piece_items(item)

                        d = pd.DataFrame({"Class":[c],
                                          "Spec":[spec],
                                          "ItemName":[item_name],
                                          "Slot":[slot],
                                          "Rank":[slot_count],
                                          "Source": [source],
                                          "Source Type": [source_type],
                                          "Website":[website]})
                        df = pd.concat([df,d])
            
#        print("Changing slots to %s"%slot_lock)
#        display(df)        
#    break

df = df.reset_index(drop=True)
df = df.drop_duplicates()

#Manually adjust some things
for i in range(df.shape[0]):
    if df.loc[i,'Source'] == "Multiple": #Handle the "Multiple" source entries
        new_source = []
        #Check if source_type contains bosses from raids
        for j in df.loc[i,"Source Type"].split(", "):
            if j in raid_boss_registry.keys():
                new_source.append(raid_boss_registry[j])
            elif "Sartharion" in j:
                new_source.append(j)
            elif "Emblem" in j:
                new_source.append("Vendor")        
        df.loc[i,'Source'] = ', '.join(new_source)
    elif df.loc[i,'ItemName'] == "Cape of the Unworthy Wizard": #Manual adjustment due to incorrect info
        df.loc[i,'Source'] = "Naxxramas (25)"
        df.loc[i,'Source Type'] = "Kel'Thuzad"
#Write the full registry
df.to_csv(expa+"/"+expa+"_list.csv",index=False) #Full file
print(expa+"_list.csv written")

#Filter to items from currently existing raids
#raid_only = df[df.Source.isin(raids)]
raid_only = df.copy()
raid_only.to_csv(expa+"/raid_BiS_"+expa+"_list.csv",index=False) #Write kara_only file
print("raid_BiS_"+expa+"_list.csv written")

#Creat a dictionary for reverse searching (item --> BiS ranking for all classes/specs)
item_dict = {}
for item in raid_only['ItemName'].unique(): #Find each unique item from raid drops (Because I'm using this during raid only)
    subset = raid_only[raid_only['ItemName']==item] #Find all entries that contain the item
    item_value = []
    for _,row in subset.iterrows(): #Parse each row of the subset
        entry = row['Spec']+"-"+row['Class']+": "+str(row['Rank']) #Combine the Spec+Class+Rank together
        item_value.append(entry) #Append
    item_dict[item] = item_value #Write the value into dictionary
    
#Write the file 
item_dict_df = pd.DataFrame({"ItemName":item_dict.keys() #Convert to pd Dataframe
             ,"Bis_Slots":item_dict.values()})

item_dict_df = item_dict_df.sort_values(by=['ItemName'])

item_dict_df.to_csv(expa+"/BiS_dictionary_"+expa+".csv",index=False) #Write
print("Dictionary written")

Reading files:
	wowwotlk\affliction-warlock.json
	wowwotlk\arcane-mage.json
	wowwotlk\arms-warrior.json
	wowwotlk\assassination-rogue.json
	wowwotlk\balance-druid.json
	wowwotlk\beast-mastery-hunter.json
	wowwotlk\blood-tank-death-knight.json
	wowwotlk\combat-rogue.json
	wowwotlk\demonology-warlock.json
	wowwotlk\destruction-fire-warlock.json
	wowwotlk\destruction-warlock.json
	wowwotlk\discipline-priest.json
	wowwotlk\elemental-shaman.json
	wowwotlk\enhancement-shaman.json
	wowwotlk\feral-dps-druid.json
	wowwotlk\feral-tank-druid.json
	wowwotlk\fire-mage.json
	wowwotlk\frost-death-knight.json
	wowwotlk\frost-mage.json
	wowwotlk\fury-warrior.json
	wowwotlk\holy-paladin.json
	wowwotlk\holy-priest.json
	wowwotlk\marksmanship-hunter.json
	wowwotlk\protection-paladin.json
	wowwotlk\protection-warrior.json
	wowwotlk\restoration-druid.json
	wowwotlk\restoration-shaman.json
	wowwotlk\retribution-paladin.json
	wowwotlk\shadow-priest.json
	wowwotlk\survival-hunter.json
	wowwotlk\unholy-death-kn

In [15]:
import sys
a = []
for class_items in all_items:
    for item in class_items:
        try:
            if item['name'] == "Ring of Decaying Beauty":
                a.append(item)
        except:
            pass
a

[{'name': 'Ring of Decaying Beauty',
  'slot': 'finger 1',
  'phase': ['T7', 'T8', 'T9', 'T10'],
  'pre-bis': {'bis': False, 'enchant': {'recipe': False}},
  't7': {'bis': False, 'enchant': {'recipe': False}},
  't8': {'bis': False, 'enchant': {'recipe': False}},
  't9': {'bis': False, 'enchant': {'recipe': False}},
  't10': {'bis': False, 'enchant': {'recipe': False}},
  'source': 'Naxxramas (25)',
  'source_type': 'Sapphiron',
  'drop_chance': 0.05,
  'rarity': 'epic'},
 {'name': 'Ring of Decaying Beauty',
  'slot': 'finger 2',
  'phase': ['T7', 'T8', 'T9', 'T10'],
  'pre-bis': {'bis': False, 'enchant': {'recipe': False}},
  't7': {'bis': False, 'enchant': {'recipe': False}},
  't8': {'bis': False, 'enchant': {'recipe': False}},
  't9': {'bis': False, 'enchant': {'recipe': False}},
  't10': {'bis': False, 'enchant': {'recipe': False}},
  'source': 'Naxxramas (25)',
  'source_type': 'Sapphiron',
  'drop_chance': 0.05,
  'rarity': 'epic'},
 {'name': 'Ring of Decaying Beauty',
  'slot':

In [5]:
def lookup(CLASS,spec,slot):    
    '''
    Function to do a quick search of BiS by class, spec, and slot
    '''
    s = df[df['Class']==CLASS][df[df['Class']==CLASS]["Spec"]==spec]
    return s[s['Slot']==slot]

lookup('mage','arcane','ranged')

,Class,Spec,ItemName,Slot,Rank,Source,Website
236,mage,arcane,Tirisfal Wand of Ascendancy,ranged,1,Karazhan,wowtbc.gg
237,mage,arcane,Nether Core's Control Rod,ranged,2,Arcatraz,wowtbc.gg
238,mage,arcane,Eredar Wand of Obliteration,ranged,3,Magtheridon's Lair,wowtbc.gg
239,mage,arcane,Voidfire Wand,ranged,4,Mana-Tombs,wowtbc.gg
240,mage,arcane,Rod of Dire Shadows,ranged,5,Quest - The Will of the Warchief,wowtbc.gg
241,mage,arcane,Nexus Torch,ranged,6,Shattered Halls,wowtbc.gg
242,mage,arcane,Gladiator's Touch of Defeat,ranged,7,PvP,wowtbc.gg
243,mage,arcane,Wand of the Seer,ranged,8,Quest - Turning Point (Scryers),wowtbc.gg


In [6]:
def bis(CLASS,spec):
    '''
    Function to find the BiS for all slots for a given Class + Spec
    '''
    slots = ['head','neck','shoulder','back','chest','hands','wrist','waist','legs','feet',
             'weapon','off hand','ranged','finger 1','finger 2','trinket 1',' trinket 2']
    
    for slot in slots:
        index = 0
        if "2" in slot:
            index = 1
        if 'trinket' in slot:
            slot = 'trinket'
        elif 'finger' in slot:
            slot = 'finger'

        slot_bis = lookup(CLASS,spec,slot).iloc[index]['ItemName']
        print("%s: %s"%(slot.capitalize(),slot_bis))

bis('priest','holy')

Head: Helm of the Fallen Defender
Neck: Archaic Charm of Presence
Shoulder: Pauldrons of the Fallen Defender
Back: Stainless Cloak of the Pure Hearted
Chest: Primal Mooncloth Robe
Hands: Gloves of Saintly Blessings
Wrist: Bands of Indwelling
Waist: Primal Mooncloth Belt
Legs: Gilded Trousers of Benediction
Feet: Boots of the Incorrupt
Weapon: Light's Justice
Off hand: Windcaller's Orb
Ranged: Blue Diamond Witchwand
Finger: Ring of Flowing Light
Finger: Violet Signet of the Grand Restorer
Trinket: Essence of the Martyr
Trinket: Ribbon of Sacrifice


In [7]:
def reverse_lookup(item_name):
    return item_dict[item_name]

reverse_lookup('Eredar Wand of Obliteration')

['affliction-warlock: 2',
 'arcane-mage: 3',
 'demonology-warlock: 3',
 'destruction fire-warlock: 2',
 'destruction-warlock: 3',
 'fire-mage: 2',
 'frost-mage: 2',
 'holy-priest: 2',
 'shadow-priest: 7']

# WowHead
Brainstorming section to play with WoWhead's registries